<a href="https://colab.research.google.com/github/jhajagos/PHR2OHDSI/blob/main/Process_OHDSI_Vocabularies_for_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json

In [2]:
%pip install pyspark==3.5.5

In [3]:
%pip install build

In [4]:
OHSDI_VOCABULARY_PATH = "/content/drive/MyDrive/OHDSI/vocabulary/20250317/"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!git clone https://github.com/jhajagos/PreparedSource2OHDSI.git

Cloning into 'PreparedSource2OHDSI'...
remote: Enumerating objects: 1303, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 1303 (delta 35), reused 68 (delta 23), pack-reused 1212 (from 1)
Receiving objects: 100% (1303/1303), 259.50 KiB | 2.18 MiB/s, done.
Resolving deltas: 100% (716/716), done.


In [8]:
%%sh
cd ./PreparedSource2OHDSI
git pull
python -m build --wheel
cd ./dist/
pip install preparedsource2ohdsi-0.1.3-py3-none-any.whl

Already up to date.
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools
  - wheel
* Getting build dependencies for wheel...
running egg_info
creating src/preparedsource2ohdsi.egg-info
writing src/preparedsource2ohdsi.egg-info/PKG-INFO
writing dependency_links to src/preparedsource2ohdsi.egg-info/dependency_links.txt
writing requirements to src/preparedsource2ohdsi.egg-info/requires.txt
writing top-level names to src/preparedsource2ohdsi.egg-info/top_level.txt
writing manifest file 'src/preparedsource2ohdsi.egg-info/SOURCES.txt'
reading manifest file 'src/preparedsource2ohdsi.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/preparedsource2ohdsi.egg-info/SOURCES.txt'
* Building wheel...
running bdist_wheel
running build
running build_py
creating build/lib/preparedsource2ohdsi
copying src/preparedsource2ohdsi/__init__.py -> build/lib/preparedsource2ohdsi
copying src/preparedsource2ohdsi/base_ohdsi_cdm.py 

In [9]:
!cd {OHSDI_VOCABULARY_PATH}; unzip *.zip

Archive:  vocabulary_download_v5_{125167fd-6b4e-4793-be48-fbf1e778cabf}_1742253362890.zip
  inflating: CONCEPT.csv             
  inflating: DOMAIN.csv              
  inflating: CONCEPT_CLASS.csv       
  inflating: CONCEPT_CPT4.csv        
  inflating: RELATIONSHIP.csv        
  inflating: DRUG_STRENGTH.csv       
  inflating: VOCABULARY.csv          
  inflating: CONCEPT_SYNONYM.csv     
  inflating: CONCEPT_ANCESTOR.csv    
  inflating: CONCEPT_RELATIONSHIP.csv  
replace cpt.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [10]:
with open(os.path.join(OHSDI_VOCABULARY_PATH, "umls_api.json")) as f:
  config= json.load(f)
umls_api_key = config["umls_api"]

In [11]:
!cd {OHSDI_VOCABULARY_PATH}; bash cpt.sh {umls_api_key}

[INFO ] 2025-03-18 11:47:14.451 [main] ConceptService - Writing updated data to CONCEPT.csv
Updated CPT4 records:  17747/17749
[INFO ] 2025-03-18 11:47:49.402 [main] ConceptService - All cpt4 concepts are processed.


In [12]:
!cd {OHSDI_VOCABULARY_PATH}; bzip2 -v *.csv

bzip2: Output file CONCEPT_ANCESTOR.csv.bz2 already exists.
bzip2: Output file CONCEPT_CLASS.csv.bz2 already exists.
bzip2: Output file CONCEPT_CPT4.csv.bz2 already exists.
bzip2: Output file CONCEPT.csv.bz2 already exists.
bzip2: Output file CONCEPT_RELATIONSHIP.csv.bz2 already exists.
bzip2: Output file CONCEPT_SYNONYM.csv.bz2 already exists.
bzip2: Output file DOMAIN.csv.bz2 already exists.
bzip2: Output file DRUG_STRENGTH.csv.bz2 already exists.
bzip2: Output file RELATIONSHIP.csv.bz2 already exists.
bzip2: Output file VOCABULARY.csv.bz2 already exists.


In [6]:
import pyspark
spark = pyspark.sql.SparkSession.builder\
    .config("spark.driver.memory", "16g") \
    .getOrCreate()

In [13]:
ohdsi_configuration = {
  "concept_base_path": OHSDI_VOCABULARY_PATH,
  "export_concept_mapping_table_path": OHSDI_VOCABULARY_PATH + "export/",
  "concept_csv_file_extension": ".csv.bz2"}
with open("/content/ohdsi_configuration.json", "w") as fw:
  json.dump(ohdsi_configuration, fw)

In [14]:
%%python /content/PreparedSource2OHDSI/map/ohdsi/build_concept_tables_for_mapping.py -c /content/ohdsi_configuration.json -l

INFO:root:Configuration:
INFO:root:{'concept_base_path': '/content/drive/MyDrive/OHDSI/vocabulary/20250317/', 'export_concept_mapping_table_path': '/content/drive/MyDrive/OHDSI/vocabulary/20250317/export/', 'concept_csv_file_extension': '.csv.bz2'}
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 11:47:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 11:47:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
INFO:root:Reading concept csv: '/content/drive/MyDrive/OHDSI/vocabulary/20250317/CONCEPT.csv.bz2'
INFO:root:Reading concept_relationship csv: '/content/drive/MyDrive/OHDSI/vocabulary/20250317/CONCEPT_RELATIONSHIP.csv.bz2'
INFO:root:Reading vocabulary csv: f'/content/drive/MyDrive/OHDSI/vocabulary/20250317/VOCABULARY.csv.bz2'
INFO:root:Exporting: 'concept' to '/content/drive/My